# Discrete Hopfield Networks

<img src="pix/hebb.png">

### Basic Rules

- Each neuron will represent 1 bit in a bit pattern.

- An n-dimensional vector will required n-neurons to represent it, e.g., (1,0,1,1,0) requires 5 neurons.

- We will represnt binary values in "bipolar" form: 1 means 1, and -1 means 0

- neurons use a fire/no fire rule based based on total input $\sum w_{ij}x_j$


we will represent three 8 bit patterns

binary(x,n) produces an n-bit vector of 1's and 0's representing the decimal integer x
bipolar(x) convertes a vector of 1's and 0's to a vectors of 1's and -1's

In [7]:
binary=function(x,n){rev(head(as.integer(intToBits(x)),n))}
bipolar=function(x){2*x-1}
frombinary=function(x){sum(c(128,64,32,16,8,4,2,1)*((x+1)/2))}

In [8]:
x1=bipolar(binary(17,8))
x2=bipolar(binary(3,8))
x3=bipolar(binary(42,8))

data=matrix(c(x1, x2, x3),ncol=3)
t(data)

-1,-1,-1,1,-1,-1,-1,1
-1,-1,-1,-1,-1,-1,1,1
-1,-1,1,-1,1,-1,1,-1


In [30]:
options(width=54)
head(as.integer(intToBits(47)),8)

[1] 1 1 1 1 0 1 0 0

In [9]:
apply(data,2,frombinary) # 2 means apply over columns

[1] 17  3 42

## Energy function - total energy of the network for any given input vector $\mathbf{x}$ - is given by

## $$\mathcal{E}=-\dfrac{1}{2}\sum x_i w_{ij} x_j = - \dfrac{1}{2} \mathbf{x}^\text{T} \mathbf{w} \mathbf{x} $$

### This form of the energy function is chosen because updates are guaranteed to decrease the total energy and because it is similar to various models used in thermodynamics.

In [16]:
energy = function(output.vector, weight.vector){-0.5 * ( output.vector %*% weight.matrix %*% output.vector)} 

## Learning rule is "neurons that fire together wire together" 

## $$\Delta  w_{ij} = \eta x_i x_j$$

## Matrix form: $$\Delta \mathbf{w} = \eta \mathbf{x} \mathbf{x}^\text{T}$$

### We can initialize the weight matrix as 

## $$\dfrac{1}{d} \sum_{k=1}^n \mathbf{x}_k \mathbf{x}^\text{T}_j$$

### $n$ is the number of training vectors and $d$ is the dimension of each vector ( = number of neurons)

In [10]:
initialize.hopfield.weights=function(data){
    n=ncol(data)
    w=0
    for (j in 1:n){
        x = data[,j]
        #
        # the following two lines add x[k]*x[ell] to every w[k,ell]
        #
        weight.updates = x %*% t(x)
        w = w+ weight.updates
    }
    w/length(x)
}

weight.matrix=initialize.hopfield.weights(data)

- ## tecnhically we should subtract the identity matrix from the weight function
- ## otherwise we have nodes that are self interacting.
- ## it doesn't really matter in the long run
- ## the only difference is that the diagonal of w will be all zero

In [14]:
weight.matrix

0.375,0.375,0.125,0.125,0.125,0.375,-0.125,-0.125
0.375,0.375,0.125,0.125,0.125,0.375,-0.125,-0.125
0.125,0.125,0.375,-0.125,0.375,0.125,0.125,-0.375
0.125,0.125,-0.125,0.375,-0.125,0.125,-0.375,0.125
0.125,0.125,0.375,-0.125,0.375,0.125,0.125,-0.375
0.375,0.375,0.125,0.125,0.125,0.375,-0.125,-0.125
-0.125,-0.125,0.125,-0.375,0.125,-0.125,0.375,-0.125
-0.125,-0.125,-0.375,0.125,-0.375,-0.125,-0.125,0.375


reference: this implementation is based on James Freman, Simulating Neural Networks with Mathematica (translated into R)

if the net input to a neuron is positive/negative, the output becomes 1/-1. If the net input is zero, then use whatever was already stored there as the output. This is the raw function inside decision. Then use mapply to evaluated this on a full vector.

In [12]:
decision = function(xvec, net){
    
    mapply(
        function(x,netin){
            if (netin > 0) 1
            else if (netin < 0) -1
            else x        
        },
        xvec,
        net)    
}

# Evaluate  Hopfield net against an input vector

In [45]:
hopfield=function(input.vector, weights){
    tol = 0.001            # convergence tolerance
    max.steps = 1000      # emergencey evacuation
    
    # initialize energy function
    
    new.energy = energy(data[,1],weights)
    
    converged = FALSE
    x=input.vector
    j=0
    while ((!converged) & (j<max.steps)) {   
        
        # calculate total input to network
        
        net.input = weights %*% x
        
        # calculate ouput of all nodes
        
        # output = phi(x,net.input)
        output = decision(x, net.input)
        
        # update energe and see if it has changed
        
        old.energy=new.energy
        new.energy = energy(output, weights)
        
        # save output as x for next iteratoin
        x=output
        
        # test for convergence
        converged = (abs(old.energy-new.energy)<tol) 
        j = j+1
        }
    print(paste("converged after j=",j," iterations;  result=",frombinary(x)))
    }

In [46]:
for (k in 15:18){hopfield(bipolar(binary(k,8)), weight.matrix)}  # these are close to 17

[1] "converged after j= 1  iterations;  result= 3"
[1] "converged after j= 1  iterations;  result= 17"
[1] "converged after j= 1  iterations;  result= 17"
[1] "converged after j= 1  iterations;  result= 3"


In [47]:
for (k in 1:4){hopfield(bipolar(binary(k,8)), weight.matrix)}  # these are close to 3

[1] "converged after j= 2  iterations;  result= 1"
[1] "converged after j= 1  iterations;  result= 3"
[1] "converged after j= 1  iterations;  result= 3"
[1] "converged after j= 2  iterations;  result= 1"


In [48]:
for (k in 39:48){hopfield(bipolar(binary(k,8)), weight.matrix)}  # these are close to 42

[1] "converged after j= 1  iterations;  result= 3"
[1] "converged after j= 2  iterations;  result= 42"
[1] "converged after j= 3  iterations;  result= 42"
[1] "converged after j= 2  iterations;  result= 42"
[1] "converged after j= 2  iterations;  result= 42"
[1] "converged after j= 3  iterations;  result= 42"
[1] "converged after j= 2  iterations;  result= 42"
[1] "converged after j= 2  iterations;  result= 42"
[1] "converged after j= 2  iterations;  result= 42"
[1] "converged after j= 1  iterations;  result= 17"


# <font color="red">Problem with Hopfield net:</font>
- # there may be multiple energy minimums
- # the network may find a local minimum and not a global minimum

# <font color="red">One solution: Stochastic Hopfield Network </font>
- # replaces discrete fire/no-fire (yes/no) choice with probability function for firing

# $$
P = \dfrac{1}{1+e^{-(\mathcal{E}/T) }}
$$

- # gradually change probability function over time to approach the discrete yes/no choice
> -  ## for large T this is a logistic function
> -  ## as $T\to 0$ this approaches a step function in the activation
- # Simulated annealing 
> - ## Start with a large T ("temperature")
> - ## Each time the network reaches an equilibrium at any given T, reduce T
> - ## Continute the process
- # Implementation
> - ## need to modify the psi function to include the probability. 

## Where to learn more: 

- ## Download and install the Stutgart Neural Network Simulator SNNS (its kind of old) but you can still play with it. http://www.ra.cs.uni-tuebingen.de/software/JavaNNS/ 
- ## it is a stand-alone simulator and does not require R or Python but it does require Java, which is always tricky
- there is a wrapper for SNNS for R that lets you call all the functions in SNNS once you get used to it